# Lab9

---

## Task

Реализовать решение ОДУ методом конечных элементов.

Сравнить решения при разных N (либо графически, либо выводить значения решений на достаточно частой сетке)

# Solution

---

In [ ]:
import numpy as np
from scipy import linalg as la
from scipy import integrate as integ

import matplotlib.pyplot as plt

### Реализация метода конечных элементов для краевой задачи

In [ ]:
def func(x, i, grid, h):
    if grid[i] < x <= grid[i + 1]:
        return (x - grid[i]) / h
    elif grid[i + 1] < x < grid[i + 2]:
        return (grid[i + 2] - x) / h
    return 0

def derivative(x, i, grid, h):
    if grid[i] < x <= grid[i + 1]:
        return 1 / h
    elif grid[i + 1] < x < grid[i + 2]:
        return - 1 / h
    return 0

def integrate(f, a, b):
    x = np.linspace(a, b, 5000)
    return integ.trapezoid(f(x), x)

# def coordinates(n, segment, order=0):
#     """
#     'order' is the order of derrivative
#     order = 0 means function itselv
#     order = 1 is the first derivative
#     """
#     valid_orders = [0, 1]
#     if order not in valid_orders:
#         raise ValueError("Wrong order")

#     a, b = segment
#     grid = np.linspace(a, b, n + 2)
#     h = grid[1] - grid[0]

#     if order == 0:
#         phi = lambda x, i: func(x, i, grid, h)
#     else:
#         phi = lambda x, i: derivative(x, i, grid, h)
    
#     phis = []
#     for i in range(n):
#         phis.append(np.frompyfunc(lambda x: phi(x, i), 1, 1))

#     return phis

def coords(n, a, b):
    grid = np.linspace(a, b, n + 2)
    h = grid[1] - grid[0]

    def phi(j):
        def f(x):
            if grid[j] < x <= grid[j + 1]:
                return (x - grid[j]) / h
            elif grid[j + 1] < x < grid[j + 2]:
                return (grid[j + 2] - x) / h
            else:
                return 0

        return f

    phis = []
    for i in range(n):
        phis.append(np.frompyfunc(phi(i), 1, 1))

    return phis


def d_coords(n, a, b):
    grid = np.linspace(a, b, n + 2)
    h = grid[1] - grid[0]

    def dphi(i):
        def f(x):
            if grid[i] < x <= grid[i + 1]:
                return 1 / h
            elif grid[i + 1] < x < grid[i + 2]:
                return - 1 / h
            else:
                return 0

        return f

    phis = []
    for i in range(n):
        phis.append(np.frompyfunc(dphi(i), 1, 1))

    return phis

def get_lbase(p, q, r, f, segment, n):
    a, b = segment
    M = np.zeros((n, n))
    phis = coords(n, a, b)
    d_phis = d_coords(n, a, b)

    for i in range(n):
        for j in range(n):
            phi_i, d_phi_i = phis[i], d_phis[i]
            phi_j, d_phi_j = phis[j], d_phis[j]
            func = lambda x: p(x) * d_phi_i(x) * d_phi_j(x) + q(x) * phi_i(x) * d_phi_j(x) + r(x) * phi_i(x) * phi_j(x)
            M[i, j] = integrate(func, a, b)

    return M

def get_rbase(p, q, r, f, segment, n):
    a, b = segment
    x = np.linspace(a, b, n + 2)
    h = x[1] - x[0]

    def each(i):
        return 1 / h * (-x[i] * integrate(f, x[i], x[i + 1]) + x[i + 2] * integrate(f, x[i + 1], x[i + 2]) + integrate(
            (lambda x: x * f(x)), x[i], x[i + 1]) - integrate(lambda x: x * f(x), x[i + 1], x[i + 2]))

    return [each(i) for i in range(n)]

def finite_elements_method(p, q, r, f, segment, n):
    a, b = segment
    phis = coords(n, a, b)
    alphas = la.solve(get_lbase(p, q, r, f, segment, n), get_rbase(p, q, r, f, segment, n))

    return lambda x: sum(alpha * phi(x) for alpha, phi in zip(alphas, phis))

## Experimental reseach

In [ ]:
def experiment(p, q, r, f, segment, n_range):
    _, axes = plt.subplots(3, 2, figsize=(20, 12))

    k = 0
    for i in range(2):
        for j in range(2):
            n = n_range[k]
            k += 1

            a, b = segment
            u = finite_elements_method(p, q, r, f, segment, n)
            grid = np.linspace(a, b, 100)

            axes[i, j].plot(grid, u(grid))
            axes[i, j].set_title(f"Number of N: {n}")

    for n in n_range:
        u = finite_elements_method(p, q, r, f, segment, n)
        axes[2, 0].plot(grid, u(grid), label=f"N = {n}")

    axes[2, 0].legend()


### Дифуры второго порядка из методички

In [ ]:
# Var 5
p = lambda x: - 1 / (x + 3)
q = lambda x: - x
r = lambda x: np.log(2 + x)
f = lambda x: 1 - x / 2

segment = (-1, 1)
experiment(p, q, r, f, segment, [2, 5, 8, 11])

In [ ]:
# Var 8
p = lambda x: -(4 - x) / (5 - 2 * x)
q = lambda x: (1 - x)/2
r = lambda x: np.log(3 + x) / 2
f = lambda x: 1 + x / 3

segment = (-1, 1)
experiment(p, q, r, f, segment, [2, 5, 8, 11])

In [ ]:
# Var 11
p = lambda x: -(7 - x)/(8 + 3 * x)
q = lambda x: (1 + x / 3)
r = lambda x: (1 - np.exp(x / 2) / 2)
f = lambda x: 1/2 - x / 3 

segment = (-1, 1)
experiment(p, q, r, f, segment, [2, 5, 8, 11])

### Дифур первого порядка


In [ ]:
p = lambda x: 0
q = lambda x: 1 / (2 + x)
r = lambda x: np.cos(x)
f = lambda x: 1 + x
segment = (-1, 1)
experiment(p, q, r, f, segment, [2, 5, 8, 11])